# RNN Poetry Play Generator

 We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

# import and setup

In [3]:
%tensorflow_version 2.x  
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

# Dataset: from shakesphere play
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')



1130496/1115394 [==============================] - 0s 0us/step


In [5]:
'''
###Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. 
Then you'll need to follow the steps from above but load in this new file instead.

'''

# load the dataset from local computer 
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

"\n###Loading Your Own Data\nTo load your own data, you'll need to upload a file from the dialog below. \nThen you'll need to follow the steps from above but load in this new file instead.\n\n"

### Read Contents of File
Let's look at the contents of the file.

In [6]:
# Read, then decode for py2 compat.
# rb: read byte mode
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')


# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))



Length of text: 1115394 characters


In [7]:
# take a look at first 250 characters in the text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encoding
Since this text isn't encoded yet,we will need to do that ourselves. We are going to encode each unique character as a different integer.

In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)} # char to int
idx2char = np.array(vocab) # int to char

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [9]:

# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
# And here we will make a function that can convert our numeric values to text.

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [11]:
# each sequence length is 100 and we are creating batch_size = 64 means at one time 64 sequences will be pass to model

seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)



In [12]:
# Next we can use the batch method to turn this stream of characters into batches of desired length


sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
# Now we need to use these sequences of length 101 and split them into input 
# and output.

def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [14]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [15]:
# Finally we need to make training batches.


BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [16]:
# while modeul build, batch_size=65 bcz at training time, we will give 64 sequence together
# but at prediciton, we just need batch_size = 1, means 1 sequence as input and 1 sequence as output


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]), # none: dont know how long sequences gonna be in each batch
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True, # want to see swquence of all input sequence
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()


## dense (Dense)--> (64, None, 65) --> we will always get 65 size at output 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch. 






However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.



In [17]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.51575252e-04  3.49325855e-04 -1.58600765e-03 ...  2.17700796e-03
   -1.36762275e-03 -3.62160592e-03]
  [ 6.40818267e-04  7.18200346e-04 -2.35996861e-03 ...  3.04595334e-03
   -2.71624932e-03 -6.77824905e-03]
  [ 7.26300990e-03 -3.62696894e-03  6.18506782e-03 ...  1.38870021e-03
   -6.80961530e-04  1.16538059e-03]
  ...
  [-9.74869728e-03 -7.79462047e-03 -8.18427012e-04 ...  5.11205697e-04
   -7.78981252e-03 -9.77268559e-04]
  [-1.13618234e-02  1.32091425e-03  3.28861922e-03 ...  1.05007843e-03
   -1.03734797e-02  6.37847744e-03]
  [-3.31222988e-03  2.61020008e-03  5.93861192e-03 ...  2.07759812e-03
   -1.30039426e-02  7.34737143e-03]]

 [[ 1.59215910e-04  3.80767114e-03 -1.05217297e-03 ...  1.25873962e-03
   -9.49652283e-04 -2.57224170e-03]
  [-2.26573576e-03  4.59675866e-05 -3.22263455e-03 ... -1.38301856e-03
   -5.89596713e-03 -3.29662394e-03]
  [-4.55075502e-03 -3.73651646e-03 -4.40997817e-03 ... -4.12227446e-03
   -9.46074072e-03 -3.75145068e-03]
  ...
  [ 8.520

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00015158  0.00034933 -0.00158601 ...  0.00217701 -0.00136762
  -0.00362161]
 [ 0.00064082  0.0007182  -0.00235997 ...  0.00304595 -0.00271625
  -0.00677825]
 [ 0.00726301 -0.00362697  0.00618507 ...  0.0013887  -0.00068096
   0.00116538]
 ...
 [-0.0097487  -0.00779462 -0.00081843 ...  0.00051121 -0.00778981
  -0.00097727]
 [-0.01136182  0.00132091  0.00328862 ...  0.00105008 -0.01037348
   0.00637848]
 [-0.00331223  0.0026102   0.00593861 ...  0.0020776  -0.01300394
   0.00734737]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.5157525e-04  3.4932586e-04 -1.5860077e-03  1.0580495e-03
 -1.0756466e-03  1.7022415e-03  1.1266479e-03 -5.1882626e-03
  1.2877248e-03 -4.1594388e-04 -1.5233394e-03  2.9064792e-03
  2.2642571e-03  6.7772967e-04 -2.1177239e-04  8.0514530e-04
 -7.8812093e-03 -1.6244564e-03 -3.7471361e-03 -6.2384885e-03
  1.4350621e-04 -4.3825572e-03  2.7045303e-03 -7.8604165e-03
  2.9901585e-03 -9.9759724e-04  4.0524909e-03 -7.9227902e-04
  2.9530510e-04 -1.1153152e-03 -1.5371351e-03 -2.0452896e-03
 -2.7586422e-03 -3.5337687e-03  6.3144456e-04  1.6130331e-03
  5.3649624e-03 -3.1710628e-03 -4.4835050e-04  4.4787587e-03
 -7.9502529e-03 -7.1591605e-04 -5.6913951e-03  1.2840227e-03
  1.3769706e-03 -2.1744159e-03 -2.1502972e-03 -7.7121379e-04
 -1.8406278e-03  3.7588554e-03  2.8732186e-03  3.2181463e-03
 -6.4059263e-03  3.1522841e-03  2.0220662e-03  2.7594540e-05
 -5.2361580e-04  1.4055714e-03 -3.0296918e-03  2.4229160e-03
  8.3761069e-04  1.4254799e-03  2.1770080e-03 -1.3676228e-03
 -3.621605

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"Tn!S.aSjjzIm\ncqwD3tLPBeBytT$RyPur\n\nolRQGuIY.ZFiHXkOoQHEGRR'oaPaYzpJRdqCpOlUWK;pLxEuYGnn&.GNt?e?VS;H:"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [23]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints

Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Training




In [25]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 31s 157ms/step - loss: 2.7336
Epoch 2/50
172/172 [==============================] - 29s 158ms/step - loss: 2.1288
Epoch 3/50
172/172 [==============================] - 29s 159ms/step - loss: 1.8829
Epoch 4/50
172/172 [==============================] - 29s 159ms/step - loss: 1.7156
Epoch 5/50
172/172 [==============================] - 29s 159ms/step - loss: 1.5970
Epoch 6/50
172/172 [==============================] - 29s 159ms/step - loss: 1.5117
Epoch 7/50
172/172 [==============================] - 29s 159ms/step - loss: 1.4499
Epoch 8/50
172/172 [==============================] - 29s 160ms/step - loss: 1.4028
Epoch 9/50
172/172 [==============================] - 29s 160ms/step - loss: 1.3641
Epoch 10/50
172/172 [==============================] - 29s 161ms/step - loss: 1.3301
Epoch 11/50
172/172 [==============================] - 29s 161ms/step - loss: 1.2995
Epoch 12/50
172/172 [==============================] - 29s 160ms/step - lo

# Prediction: Generate Text

### **Loading the Model (with epoch=1)**
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.



In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

### **Save checkpoint to store weights**

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.


In [30]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # to get latest checkpoints
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [31]:
# to get intermediate checkpoints

# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

### now Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: juliet
julieth us beat for you.
He said on the embracements of worse,
And let missey hide the right: saw him all of fifesters. Let this great loathsome?
Our daughter,
Both him have been absolver. All men all
Here of Angelo's shape, that have the fires seath'd hast
Should promise obsequising steel, and shall proceed balladion and our woes
We hence to forecond us three-sensible hours
And pluck your instruction. Good father.

BIONDELLO:
I fear me but stop bare a guistless skill
Be some more duty whether the very pin,
Or I have famed to Harry that to you.
See, enter, I'l very rich, but that your word s rid it. Then have you well do
what
laid fast for me to curse
Abides in me; I will keep to my heir
And from this king is victory.

ISABELLA:
O desperate, when I say and Drightful censure
I live in banishment 
